In [1]:
import os
from pathlib import Path
import shutil

import teehr

teehr.__version__

'0.6.0dev2'

In [2]:
from teehr.evaluation.spark_session_utils import create_spark_session

In [3]:
spark = create_spark_session(
    aws_access_key_id="minioadmin",
    aws_secret_access_key="minioadmin123"
)

INFO:teehr.evaluation.spark_session_utils:🚀 Creating Spark session: TEEHR Evaluation
INFO:teehr.evaluation.spark_session_utils:✅ Spark local configuration successful!
INFO:teehr.evaluation.spark_session_utils:🔑 Using user-provided AWS credentials
INFO:teehr.evaluation.spark_session_utils:   - Using long-term credentials
INFO:teehr.evaluation.spark_session_utils:Configuring Iceberg catalogs...
INFO:teehr.evaluation.spark_session_utils:⚠️  Configuring remote catalog for MinIO access
INFO:teehr.evaluation.spark_session_utils:Spark session created for TEEHR Evaluation.
INFO:teehr.evaluation.spark_session_utils:🎉 Spark session created successfully!


In [4]:
%%time
dir_path = "/data/temp_warehouse"

ev = teehr.Evaluation(
    spark=spark,
    dir_path=dir_path,
    create_dir=True
)

INFO:teehr.evaluation.evaluation:Directory /data/temp_warehouse already exists. Not creating it again.
INFO:teehr.evaluation.evaluation:Using provided Spark session.
INFO:teehr.evaluation.evaluation:Active catalog set to local.


CPU times: user 346 μs, sys: 3.84 ms, total: 4.19 ms
Wall time: 57.8 ms


In [5]:
ev.set_active_catalog("remote")

ev.active_catalog

INFO:teehr.evaluation.evaluation:Active catalog set to remote.


RemoteCatalog(warehouse_dir='s3://warehouse/', catalog_name='iceberg', namespace_name='teehr', catalog_type='rest', catalog_uri='http://iceberg-rest:8181')

In [6]:
from teehr import DeterministicMetrics as dm
from teehr import SignatureMetrics as sm

In [7]:
sdf = ev.metrics.query(
    group_by=["primary_location_id", "configuration_name"],
    include_metrics=[
        sm.Count(),
        sm.Average(),
        dm.RelativeBias(),
        dm.NashSutcliffeEfficiency(),
        dm.KlingGuptaEfficiency()
    ]
).to_sdf()

INFO:teehr.evaluation.tables.base_table:Loading files from iceberg.teehr.joined_timeseries.
INFO:teehr.evaluation.read:Reading files from iceberg.teehr.joined_timeseries.
INFO:teehr.evaluation.metrics:Calculating performance metrics.


In [8]:
sdf.createTempView("metrics")

In [15]:
sdf = ev.spark.sql("""
SELECT m.*, l.* FROM metrics m JOIN iceberg.teehr.locations l ON l.id = m.primary_location_id
""")
sdf = sdf.drop("id")

In [16]:
sdf.show()

+-------------------+-------------------+--------+---------+-------------+-------------------------+----------------------+--------------------+--------------------+
|primary_location_id| configuration_name|   count|  average|relative_bias|nash_sutcliffe_efficiency|kling_gupta_efficiency|                name|            geometry|
+-------------------+-------------------+--------+---------+-------------+-------------------------+----------------------+--------------------+--------------------+
|      usgs-08313000|nwm30_retrospective|266814.0|35.494263|    2.0169313|               -16.590868|            -2.7586813|RIO GRANDE AT OTO...|[01 01 00 00 00 E...|
|      usgs-11421000|nwm30_retrospective|267701.0|57.265133|   0.42592105|                0.6365238|            0.54856336|YUBA R NR MARYSVI...|[01 01 00 00 00 7...|
|      usgs-01347000|nwm30_retrospective|244887.0|88.383896| -0.095626995|               0.53044313|             0.6748171|MOHAWK RIVER NEAR...|[01 01 00 00 00 3...|
|   

In [17]:
ev.write.to_warehouse(source_data=sdf, table_name="sim_metrics_by_location", write_mode="create_or_replace")